# HW 7 Рекомендательные системы

## Получение датасета 

In [20]:
import json
import gzip
import numpy as np
import pandas as pd

# Библиотеки по машинному обучению
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

# Библиотеки построения диаграмм
from matplotlib import pyplot as plt
import seaborn as sns

# Установка режима отображения диаграмм
%matplotlib inline
plt.rcParams["figure.figsize"] = (18, 5)
plt.style.use("ggplot")

Берем данные по продаже техники по ссылке https://nijianmo.github.io/amazon/index.html, стоит обратить внимание на раздел "Small" subsets for experimentation, где представлены не самые большие датасеты (чтобы и в память поместился, и считался недолго)

In [21]:
# Функция чтения данных из файла gzip - json
def parse(path):
    for line in gzip.open(path, 'rb'):
        yield json.loads(line)

# Функция загрузки данных из файла json в датафрейм
def get_dataframe(path):
    data = {}
    for i, item in enumerate(parse(path)):
        data[i] = item
        
    return pd.DataFrame.from_dict(data, orient='index')

In [137]:
# Загрузка данных по рейтингу продаж техники
# asin - идентификатор продукта, например, 111846130
# reviewer_id - идентификатор рецензента, например, A3NHUQ33CFH3VM 
# rating - рейтинг
# timestamp - время отзыва
rating_data = pd.read_csv('Appliances.csv', names=['asin', 'reviewer_id', 'rating', 'timestamp'])
print(len(rating_data))
rating_data.head()

602777


,asin,reviewer_id,rating,timestamp
0,1118461304,A3NHUQ33CFH3VM,5.0,1385510400
1,1118461304,A3SK6VNBQDNBJE,5.0,1383264000
2,1118461304,A3SOFHUR27FO3K,5.0,1381363200
3,1118461304,A1HOG1PYCAE157,5.0,1381276800
4,1118461304,A26JGAM6GZMM4V,5.0,1378512000


In [136]:
# Загрузка данных о товаре
#asin - идентификатор продукта, например. 0000013714
#title - описание товара
#brand - брэнд
#main_cat - категория
#price - цена
meta_data = get_dataframe('meta_Appliances.json.gz')
print(len(meta_data))
meta_data.head()

30445


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[],,Tupperware Freezer Square Round Container Set ...,[],,Tupperware,[Each 3-pc. set includes two 7/8-cup/200 mL an...,"[>#39,745 in Appliances (See top 100)]",[],{},Appliances,,"November 19, 2008",,7301113188,[],[]
1,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[2 X Tupperware Pure & Fresh Unique Covered Co...,,2 X Tupperware Pure &amp; Fresh Unique Covered...,[],,Tupperware,[2 X Tupperware Pure & Fresh Unique Covered Co...,"[>#6,118 in Appliances (See top 100)]",[B004RUGHJW],{},Appliances,,"June 5, 2016",$3.62,7861850250,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,"[Appliances, Parts &amp; Accessories]",,[],,The Cigar - Moments of Pleasure,[],,The Cigar Book,[],"[>#1,861,816 in Home &amp; Kitchen (See Top 10...","[B01HCAVSLK, 1632206579]",{},Amazon Home,,,$150.26,8792559360,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,"[Appliances, Parts & Accessories]","class=""a-keyvalue prodDetTable"" role=""present...","[Multi purpost descaler, especially suited to ...",,Caraselle 2X 50G Appliance Descalene,[],,Caraselle,[],"[>#1,654,505 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"December 17, 2014",.a-box-inner{background-color:#fff}#alohaBuyBo...,9792954481,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,"[Appliances, Parts & Accessories, Range Parts ...","class=""a-keyvalue prodDetTable"" role=""present...",[Full gauge and size beveled-edge; furnished w...,,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,[],,EATON Wiring,[Returns will not be honored on this closeout ...,"[>#3,066,990 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"January 16, 2007",$3.43,B00002N5EL,[],[]


In [153]:
# Объединяем датасеты
data = rating_data.merge(meta_data[['asin', 'title', 'brand', 'main_cat', 'price']].drop_duplicates(), on='asin', how='left')
data.columns = ['item_id', 'user_id', 'rating', 'timestamp','title', 'brand', 'main_cat', 'price']
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data = data[data['title'].isnull() == False].reset_index(drop=True)
data.head()

,item_id,user_id,rating,timestamp,title,brand,main_cat,price
0,B00002N7IL,A3SHVDMM83IHJ4,5.0,2015-03-20,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...",Leviton,Tools & Home Improvement,
1,B00004SQHD,A2OXDRWBASV91Y,5.0,2013-04-20,Coleman Cable 09045 5-Foot Range Cord,Coleman Cable,Tools & Home Improvement,$1.41
2,B00004SQHD,A2KG6AWJSWILPR,5.0,2013-03-16,Coleman Cable 09045 5-Foot Range Cord,Coleman Cable,Tools & Home Improvement,$1.41
3,B00004SQHD,A2CBE6VYOARZN4,5.0,2017-02-14,Coleman Cable 09045 5-Foot Range Cord,Coleman Cable,Tools & Home Improvement,$1.41
4,B00004SQHD,AVKOTZD5ZIOX5,5.0,2016-12-21,Coleman Cable 09045 5-Foot Range Cord,Coleman Cable,Tools & Home Improvement,$1.41


# EDA

In [154]:
# Вывод информации о структуре данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601630 entries, 0 to 601629
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   item_id    601630 non-null  object        
 1   user_id    601630 non-null  object        
 2   rating     601630 non-null  float64       
 3   timestamp  601630 non-null  datetime64[ns]
 4   title      601630 non-null  object        
 5   brand      601630 non-null  object        
 6   main_cat   601630 non-null  object        
 7   price      601630 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 36.7+ MB


In [155]:
# Список товаров и кол-во их покупок
data.item_id.value_counts()

B000AST3AK    6510
B004UB1O9Q    5702
B0014CN8Y8    4048
B00KJ07SEM    3200
B0045LLC7K    2936
              ... 
B00JNBQ76I       1
B00D8NZ47A       1
B00UUELJ2E       1
B0093R9836       1
B00ZRYHEKI       1
Name: item_id, Length: 30238, dtype: int64

In [156]:
# Список покупателей и кол-во их покупок
data.user_id.value_counts()

A8WEXFRWX1ZHH     208
A1IT56MV1C09VS    207
A2LDP3A4IE9T6T    206
A1WD61B0C3KQZB    206
A25C30G90PKSQA    206
                 ... 
A3KUUR06Z5KF4M      1
AOMG5Y61KBK33       1
AZBTDQNFH1Y30       1
A3IPHTNWGBMI5A      1
A2MMETCOM8I7RS      1
Name: user_id, Length: 514945, dtype: int64

In [157]:
# Спискок рейтингов
data.rating.value_counts()

5.0    415598
4.0     75260
1.0     59552
3.0     30551
2.0     20669
Name: rating, dtype: int64

### Предобработка данных

In [158]:
# Исключение непопулярных товаров и покупателей, кто купил мало товаров
df = data.copy()
df = df[df.groupby('item_id').transform('count').iloc[:, 0] > 20]
df = df[df.groupby('user_id').transform('count').iloc[:, 0] > 2]

# Удаление пустых позиций из датафрейма
df = df.drop_duplicates(['item_id', 'user_id']).dropna()
df.shape

(30950, 8)

In [166]:
# Функция разбиения данных на тренировочную и тестовую выборки
def train_test_split(data, ratio=0.2, user_col='user_id', item_col='item_id',
                     rating_col='rating', time_col='timestamp'):
    data = data.sort_values(by=[time_col])
    
    indx = int(len(data) * (1 - ratio))
    train_data = data[[user_col, item_col, rating_col]][:indx]
    test_data = data[[user_col, item_col, rating_col]][indx:]
    
    return train_data, test_data

In [180]:
train_data, test_data = train_test_split(df, ratio=0.2)
print(train_data.shape, test_data.shape)

(24760, 3) (6190, 3)


In [181]:
train_data.head()

,user_id,item_id,rating
3413,AWGBKGKCJP4S6,B0000AYKNT,5.0
5738,A3JUHAXJ8FJMKH,B0000WM2IG,5.0
3360,A13ZAE474OMCKJ,B0000AYKNT,5.0
9938,A13ZAE474OMCKJ,B0002KXMT4,5.0
58020,A1X696GX2VPV5C,B0011YOEGA,4.0


### User-based model

In [182]:
class UserBased(BaseEstimator):
    def fit(self, train_data, user_col='user_id', item_col='item_id', rating_col='rating'):
        data = train_data.copy()
        
        # Определение списка покупателей и товаров
        self.users = data[user_col].unique()
        self.items = data[item_col].unique()

        # Определение среднего рейтинга товара для каждого покупателя
        self.rating_mean = data.groupby(user_col)[rating_col].mean()
        
        # Расчет среднего рейтинга товара для каждого покупателя в датафрейме
        data[rating_col] = data[rating_col] - data.groupby(user_col)[rating_col].transform('mean')

        # Построение сводной таблицы рейтингов товаров в разрезе покупателя и товара
        self.ratings = pivot_table(data, values=rating_col, index=user_col, columns=item_col)

        # Расчет коэффициента схожести покупателей
        self.similarities = pd.DataFrame(cosine_similarity(self.ratings), index=self.ratings.index)
        
        return self

    def predict_rating(self, pr_user, pr_item):
        # если в обучающей выборке нет такого товара или покупателя то устнавливаем нулевой рейтинг
        if not pr_item in self.items or not pr_user in self.users:
            return 0

        # Расчет прогнозного значения рейтинга товара на основе коэффициента схожести
        numerator = np.dot(self.similarities.loc[pr_user], self.ratings.loc[:, pr_item])
        denominator = self.similarities.loc[pr_user].sum() - 1
        
        return self.rating_mean[pr_user] + numerator / denominator
    

    def predict(self, X, user_col='user_id', item_col='item_id'):
        # Расчет прогнозных значений рейтингов товаров
        rating_pred = test_data[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return rating_pred

In [183]:
# Функция построения сводных таблиц
def pivot_table(data, values='rating', index='user_id', columns='item_id'):
    rows, rows_pos = np.unique(data[index], return_inverse=True)
    cols, cols_pos = np.unique(data[columns], return_inverse=True)
    
    table = np.zeros((len(rows), len(cols)), dtype=np.float16)
    table[rows_pos, cols_pos] = data[values]
    
    return pd.DataFrame(table, index=rows, columns=cols)

# Функция расчета коэффициента RMSE
def RMSE(Y_true, Y_pred):
    return np.sqrt(mean_squared_error(Y_true, Y_pred))

In [184]:
# Построение модели и прогнозирование рейтинга товаров на основе схожести покупателей
print('Старт обучения UserBased...')
ub_model = UserBased().fit(train_data)
print('Старт прогнозирования...')
ub_pred = ub_model.predict(test_data)
print('RMSE = {:0.4f}'.format(RMSE(test_data['rating'], ub_pred)))

Старт обучения UserBased...
Старт прогнозирования...
RMSE = 3.2899


### Item-based model

In [131]:
class ItemBased(BaseEstimator):
    def fit(self, train_data, user_col='user_id', item_col='item_id', rating_col='rating'):
        data = train_data.copy()

        # Определение списка покупателей и товаров
        self.users = data[user_col].unique()
        self.items = data[item_col].unique()
        
        # Определение среднего рейтинга для каждого товара
        self.rating_mean = data.groupby(item_col)[rating_col].mean()

        # Расчет среднего рейтинга каждого товара в датафрейме
        data[rating_col] = data[rating_col] - data.groupby(item_col)[rating_col].transform('mean')

        # Построение сводной таблицы рейтингов товаров в разрезе покупателя и товара
        self.ratings = pivot_table(data, values=rating_col, index=item_col, columns=user_col)

        # Расчет коэффициента схожести товаров
        self.similarities = pd.DataFrame(cosine_similarity(self.ratings), index=self.ratings.index)

        return self
    
    def predict_rating(self, pr_user, pr_item):
        # Для отсутствующих покупателей и товаров устанавливаем нулевой рейтинг
        if not pr_item in self.items or not pr_user in self.users:
            return 0
        
        # Расчет прогнозного значения рейтинга товара на основе коэффициента схожести
        numerator = np.dot(self.similarities.loc[pr_item], self.ratings.loc[:, pr_user])
        denominator = self.similarities.loc[pr_item].sum() - 1
        
        return self.rating_mean[pr_item] + numerator / denominator
    
    def predict(self, test_data, user_col='user_id', item_col='item_id'):
        # Расчет прогнозных значений рейтингов товаров
        rating_pred = test_data[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return rating_pred

In [132]:
# Построение модели и прогнозирование рейтинга товаров на основе схожести товаров
print('Старт обучения ItemBased...')
ib_model = ItemBased().fit(train_data)
print('Старт прогнозирования...')
ib_pred = ib_model.predict(test_data)
print('RMSE = {:0.4f}'.format(RMSE(test_data['rating'], ib_pred)))

Старт обучения ItemBased...
Старт прогнозирования...
RMSE = 3.2595


### Проверка рекомендательной системы

In [197]:
item_titles = data.groupby(['item_id', 'title','brand', 'main_cat', 'price'])['timestamp'].max().reset_index()
item_titles

,item_id,title,brand,main_cat,price,timestamp
0,7301113188,Tupperware Freezer Square Round Container Set ...,Tupperware,Appliances,,2009-03-13
1,7861850250,2 X Tupperware Pure &amp; Fresh Unique Covered...,Tupperware,Appliances,$3.62,2017-01-14
2,8792559360,The Cigar - Moments of Pleasure,The Cigar Book,Amazon Home,$150.26,2016-12-30
3,9792954481,Caraselle 2X 50G Appliance Descalene,Caraselle,Tools & Home Improvement,.a-box-inner{background-color:#fff}#alohaBuyBo...,2016-10-16
4,B00002N5EL,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,EATON Wiring,Tools & Home Improvement,$3.43,2016-01-21
...,...,...,...,...,...,...
30233,B01HJH651Y,Bosch 00494772 Sealing,Bosch,Tools & Home Improvement,$20.01,2018-07-23
30234,B01HJH6JT2,Bosch 00642855 Sensor,Bosch,Tools & Home Improvement,$49.33,2018-03-22
30235,B01HJH92JQ,Bosch 00175338 Switch,Bosch,Tools & Home Improvement,$40.23,2018-03-23
30236,B01HJHHEA0,Frigidaire 316543810 Knob,Frigidaire,Tools & Home Improvement,$14.99,2018-08-01


In [198]:
# Случайно выбранный покупатель для построения системы рекомендаций
user_id = 'A3NHUQ33CFH3VM'

# Построение вектора покупателя
user_vector = pivot_table(df).loc[user_id].reset_index()
user_vector['user_id'] = user_id

# Переименование столбцов датафрейма и задание нужной последовательности
user_vector.columns = ['item_id', 'rating', 'user_id']
user_vector = user_vector[['user_id', 'item_id', 'rating']]
user_vector = user_vector.merge(item_titles, how='left', on='item_id')

# Вывод первых позиций
user_vector.head()

# Вроде все, как-то связано с техникой для увлажнения воздуха

,user_id,item_id,rating,title,brand,main_cat,price,timestamp
0,A3NHUQ33CFH3VM,B00004SQHD,0.0,Coleman Cable 09045 5-Foot Range Cord,Coleman Cable,Tools & Home Improvement,$1.41,2018-04-30
1,A3NHUQ33CFH3VM,B00004YWK2,0.0,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...",Dundas Jafine,Tools & Home Improvement,$9.90,2018-02-03
2,A3NHUQ33CFH3VM,B000056J8D,0.0,"Protec DynaFilter Humidifier Cartridge, Air Cl...",Pro Tec,Amazon Home,,2018-03-16
3,A3NHUQ33CFH3VM,B00005OU6T,0.0,"Holmes &quot;C&quot; Humidifier Filter, HWF65P...",Holmes,Amazon Home,$5.98,2018-04-29
4,A3NHUQ33CFH3VM,B00006IV17,0.0,"Holmes &quot;A&quot; Humidifier Filter, HWF62",Holmes,Amazon Home,$11.89,2018-05-02


### Выводы

Были рассмотрены два типа рекомендательных систем:

Коллоборативная фильтрация (Colloborative filtering) основанная на сравнение текущего пользователя с другими и
Контентная фильтрация (Content-based filtering) - предсказания, основываясь на информации о пользователе и о том контенте, который он потребил.

Не были рассмотрены другие подходы, такие как SVD, который использует возможность сингулярного разложения матрицы пользователь-предмет, что позволяет выявлять какие-то скрытые параметры, например, как пол влияет на рейтинг и т.д.
А также контентную фильтрации и гибридные подходы.

Тему рекомендательных систем, хотелось бы продолжить более углубленно на курсовой работе.